[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_toolbox/pdftools_toolbox_update_annotations.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDKXT/latest/pdftools_toolbox-latest.tar.gz
%pip install ipython

# Update annotations to PDF
Remove the 'Ellipse' annotations from the PDF and export
the new list of annotations to a new FDF-File.

In [ ]:
import io
from pdftools_toolbox.pdf import CopyStrategy, Document, FileReference, Metadata, PageCopyOptions, Page
from pdftools_toolbox.pdf.content import IccBasedColorSpace
from pdftools_toolbox.pdf.annotations import Annotation, EllipseAnnotation
from pdftools_toolbox.pdf.navigation import ViewerSettings

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/GraphicsWhiteTextNoTP.pdf'
input_fdf_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/GraphicsWhiteTextNoTP.fdf'
input_file_path = 'GraphicsWhiteTextNoTP.pdf'
input_fdf_path = 'GraphicsWhiteTextNoTP.fdf'
prepare_file(input_url, input_file_path)
prepare_file(input_fdf_url, input_fdf_path)
output_file_path = 'UpdatedAnnotations.pdf'
output_fdf_path = 'UpdatedAnnotations.fdf'

In [ ]:
def copy_document_data(in_doc: Document, out_doc: Document):
    # Copy document-wide data

    # Output intent
    if in_doc.output_intent is not None:
        in_doc.output_intent = IccBasedColorSpace.copy(out_doc, in_doc.output_intent)

    # Metadata
    out_doc.metadata = Metadata.copy(out_doc, in_doc.metadata)

    # Viewer settings
    out_doc.viewer_settings = ViewerSettings.copy(out_doc, in_doc.viewer_settings)

    # Associated files (for PDF/A-3 and PDF 2.0 only)
    outAssociatedFiles = out_doc.associated_files
    for in_file_ref in in_doc.associated_files:
        outAssociatedFiles.append(FileReference.copy(out_doc, in_file_ref))

    # Plain embedded files
    out_embedded_files = out_doc.plain_embedded_files
    for in_file_ref in in_doc.plain_embedded_files:
        out_embedded_files.append(FileReference.copy(out_doc, in_file_ref))

In [ ]:
def filter_annotations(in_doc: Document, out_doc: Document):
    """Filter annotations and remove 'Ellipse' annotations."""
    # Define page copy options
    copy_options = PageCopyOptions()
    # Remove all annotations: we will add the filtered ones later
    copy_options.annotations = CopyStrategy.REMOVE

    for in_page in in_doc.pages:
        # Copy page to the output document
        out_page = Page.copy(out_doc, in_page, copy_options)

        # Hold the annotations from the input document
        in_annotations = in_page.annotations

        # Selectively copy annotations (excluding EllipseAnnotations - like Circle)
        for in_annotation in in_annotations:
            if not isinstance(in_annotation, EllipseAnnotation):
                out_page.annotations.append(Annotation.copy(out_doc, in_annotation))

        # Add the page to the output document
        out_doc.pages.append(out_page)

In [ ]:
try:
    # Set and check license key. If the license key is not valid, an exception is thrown.
    from pdftools_toolbox.sdk import Sdk
    Sdk.initialize("INSERT-LICENSE-KEY", None)

    # Open input PDF and FDF files
    with io.FileIO(input_file_path, "rb") as in_stream:
        with io.FileIO(input_fdf_path, "rb") as in_fdf_stream:
            with Document.open_with_fdf(in_stream, in_fdf_stream, None) as in_doc:
                # Create output PDF and FDF files
                with io.FileIO(output_file_path, "wb+") as out_stream:
                    with io.FileIO(output_fdf_path, "wb+") as out_fdf_stream:
                        with Document.create_with_fdf(out_stream, out_fdf_stream, in_doc.conformance, None) as out_doc:
                            # Copy document-wide data
                            copy_document_data(in_doc, out_doc)
    
                            # Filter and process annotations
                            filter_annotations(in_doc, out_doc)

    print("Execution successful.")
except Exception as e:
    print(f"An error occurred: {e}")